In [42]:
import nltk
from nltk.corpus import brown
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from nltk.stem import WordNetLemmatizer
%pylab inline
from sklearn.cross_validation import train_test_split
import numpy.random as npr
from sklearn.feature_extraction.text import TfidfTransformer

### PART B preparation

# drop non-string fields in df, combine each row into one string
def str_text():
    # use pandas read_csv instead of basic form read()!
    text_df = pd.read_csv('Train_15000.csv')
    # drop SalaryNormalized and save it in a new variable
    salary = text_df.pop('SalaryNormalized')
    # drop columns we don't want
    text_df = text_df.drop('Id', axis=1)
    text_df = text_df.drop('SalaryRaw', axis=1)
    # write a function to convert a row into a string
    def to_str(row):
        return ' '.join(pd.Series(row.values).map(str))
    text_str = text_df.apply(to_str, axis=1)
    text_str = pd.Series([text.decode('ascii', 'ignore') for text in text_str])
    return text_str, salary

    

# B1 -- basic multinomial bayes prediction
def prediction():
    text_str = str_text()[0]
    # get the bag of words
    vectorizer = CountVectorizer(min_df=1)
    X = vectorizer.fit_transform(text_str)
    X = X.toarray()

    # mask the salary into high (75 percentile and above) and low (below that)
    salary = str_text()[1]
    cutoff = salary.quantile(q=.75)
    y = (salary >= cutoff)
    
    # resample the high salary to make equal sample of high and low salary
    n = len(y[y==0])
    index_high = [i for i in y.index if y[i]]
    index_low = [i for i in y.index if not y[i]]
    new_high = npr.choice(index_high, size = n, replace = True)
    high_X = X[new_high]
    high_y = y[new_high]
    low_X = X[index_low]
    low_y = y[index_low]
    new_X = np.concatenate((low_X, high_X), axis=0)
    new_y = np.concatenate((low_y, high_y), axis = 0)
    

    # split into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.4, random_state=42)

    # fit the model and predict
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    print('Multinomial prediction:')
    return conf_matrix, accuracy

# B2: bayes with lemmatization, B3: bayes without stopwords
def prediction_lemma():
    text_str = str_text()[0]

    # use lemmatization. My guess is it will improve the result but not much, since 92% is already very good.
    wordnet_lemmatizer = WordNetLemmatizer()

    # first split rows into tokens and lemmatize each of them
    def lemma(row):
        text_token = nltk.word_tokenize(row)
        return [wordnet_lemmatizer.lemmatize(word) for word in text_token]
    lemmatized = text_str.apply(lemma)
    # and then merge tokens back into strings, and then put strings into a series, because fit_transform() only takes series
    lemmatized_str = pd.Series([' '.join(row) for row in lemmatized])

    # construct X sparse matrix as usual
    vectorizer = CountVectorizer(min_df=1)
    X = vectorizer.fit_transform(lemmatized_str)
    X = X.toarray()

    # construct y vector as usual. mask the salary into high (75 percentile and above) and low (below that)
    salary = str_text()[1]
    cutoff = salary.quantile(q=.75)
    y = (salary >= cutoff)

    # resample the high salary to make equal sample of high and low salary
    n = len(y[y==0])
    index_high = [i for i in y.index if y[i]]
    index_low = [i for i in y.index if not y[i]]
    new_high = npr.choice(index_high, size = n, replace = True)
    high_X = X[new_high]
    high_y = y[new_high]
    low_X = X[index_low]
    low_y = y[index_low]
    new_X = np.concatenate((low_X, high_X), axis=0)
    new_y = np.concatenate((low_y, high_y), axis = 0)
    
    # split into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.4, random_state=42)

    # fit the model and predict
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    print('Multinomial prediction with lemmatization:')
    print(conf_matrix, accuracy)

    # continue with removing stopwords. B3.
    def stop(row):
        return [word for word in row if word not in stopwords.words('english')]
    no_stopword = lemmatized.map(stop)
    no_stopword_str = pd.Series([' '.join(row) for row in no_stopword])

    vectorizer = CountVectorizer(min_df=1)
    X = vectorizer.fit_transform(no_stopword_str)
    X = X.toarray()
    
    # resample the high salary to make equal sample of high and low salary
    n = len(y[y==0])
    index_high = [i for i in y.index if y[i]]
    index_low = [i for i in y.index if not y[i]]
    new_high = npr.choice(index_high, size = n, replace = True)
    high_X = X[new_high]
    high_y = y[new_high]
    low_X = X[index_low]
    low_y = y[index_low]
    new_X = np.concatenate((low_X, high_X), axis=0)
    new_y = np.concatenate((low_y, high_y), axis = 0)

    # split into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.4, random_state=42)

    # fit the model and predict
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    # removing stopwords slightly hurt the result.
    print('Multinomial prediction without stopwords:')
    print(conf_matrix, accuracy)

    # show the top 10 indicative words
    # construct a series with negative class (low salary) features and probabilities, sort it, get the first 10
    no_prob = pd.Series(data=clf.feature_log_prob_[0], index=vectorizer.get_feature_names())
    no_prob.sort(ascending=False)
    print('top 10 indicative words for low salary class:')
    print(no_prob[:10])

    # high salary features
    yes_prob = pd.Series(data=clf.feature_log_prob_[1], index=vectorizer.get_feature_names())
    yes_prob.sort(ascending=False)
    print('top 10 indicative words for high salary class:')
    print(yes_prob[:10])

from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

# B4
def bigram():
    text = str_text()

    # this is a function I adapted from http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/
    # basically it takes in a string of tokenized words (one job description = one string),
    # creates all possible bigrams, and finds the most important bigrams based on a score (feature selection!),
    # and returns a string of all unigrams (= words) and bigrams
    def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=100):
        bigram_finder = BigramCollocationFinder.from_words(words)
        bigrams = bigram_finder.nbest(score_fn, n)
        # make sure all elements in the list are strings instead of lists, otherwise join() won't take it
        bigrams = [str(b) for b in bigrams]
        words = [str(w) for w in words]
        # merge two lists
        words = words + bigrams
        # return a string of bigrams and unigrams
        return ' '.join(words)

    ### IT'S ALL ABOUT DATA STRUCTURE AND DATA TYPE...
    # WHAT FEEDS INTO THE X IN FINAL MODEL? a series, and each element should contain all unigrams and bigrams
    # so that's what we're gonna do. bigram_word_feats() builds the elements for us, we have to put them into a series now
    all_rows_list = []
    # the data structure of text is: a tuple, length is 2, element 1 is a series of all job descriptions, element 2 is all salaries
    for i in range(len(text[0])):
        row_tokenized = nltk.word_tokenize(text[0][i])
        liststr = bigram_word_feats(row_tokenized)
        all_rows_list.append(liststr)
    all_rows_series = pd.Series(all_rows_list)

    # build the X matrix
    vectorizer = CountVectorizer(min_df=1)
    X = vectorizer.fit_transform(all_rows_series)
    
    
    X = X.toarray()

    # build y by masking the salary into high (75 percentile and above) and low (below that)
    salary = str_text()[1]
    cutoff = salary.quantile(q=.75)
    y = (salary >= cutoff)
    
    n = len(y[y==0])
    index_high = [i for i in y.index if y[i]]
    index_low = [i for i in y.index if not y[i]]
    new_high = npr.choice(index_high, size = n, replace = True)
    high_X = X[new_high]
    high_y = y[new_high]
    low_X = X[index_low]
    low_y = y[index_low]
    new_X = np.concatenate((low_X, high_X), axis=0)
    new_y = np.concatenate((low_y, high_y), axis = 0)

    # split into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.4, random_state=42)

    # fit the model and predict
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    print('Multinomial prediction with bigram:')
    return conf_matrix, accuracy


def tf_idf():
    text = str_text()

    # this is a function I adapted from http://streamhacker.com/2010/05/24/text-classification-sentiment-analysis-stopwords-collocations/
    # basically it takes in a string of tokenized words (one job description = one string),
    # creates all possible bigrams, and finds the most important bigrams based on a score (feature selection!),
    # and returns a string of all unigrams (= words) and bigrams
    def bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=100):
        bigram_finder = BigramCollocationFinder.from_words(words)
        bigrams = bigram_finder.nbest(score_fn, n)
        # make sure all elements in the list are strings instead of lists, otherwise join() won't take it
        bigrams = [str(b) for b in bigrams]
        words = [str(w) for w in words]
        # merge two lists
        words = words + bigrams
        # return a string of bigrams and unigrams
        return ' '.join(words)

    ### IT'S ALL ABOUT DATA STRUCTURE AND DATA TYPE...
    # WHAT FEEDS INTO THE X IN FINAL MODEL? a series, and each element should contain all unigrams and bigrams
    # so that's what we're gonna do. bigram_word_feats() builds the elements for us, we have to put them into a series now
    all_rows_list = []
    # the data structure of text is: a tuple, length is 2, element 1 is a series of all job descriptions, element 2 is all salaries
    for i in range(len(text[0])):
        row_tokenized = nltk.word_tokenize(text[0][i])
        liststr = bigram_word_feats(row_tokenized)
        all_rows_list.append(liststr)
    all_rows_series = pd.Series(all_rows_list)

    # build the X matrix
    vectorizer = CountVectorizer(min_df=1)
    X = vectorizer.fit_transform(all_rows_series)
    X = X.toarray()
    tfidf_transformer = TfidfTransformer()
    X = tfidf_transformer.fit_transform(X)
    X = X.toarray()

    # build y by masking the salary into high (75 percentile and above) and low (below that)
    salary = str_text()[1]
    cutoff = salary.quantile(q=.75)
    y = (salary >= cutoff)
    
    n = len(y[y==0])
    index_high = [i for i in y.index if y[i]]
    index_low = [i for i in y.index if not y[i]]
    new_high = npr.choice(index_high, size = n, replace = True)
    high_X = X[new_high]
    high_y = y[new_high]
    low_X = X[index_low]
    low_y = y[index_low]
    new_X = np.concatenate((low_X, high_X), axis=0)
    new_y = np.concatenate((low_y, high_y), axis = 0)

    # split into training and test dataset
    X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.4, random_state=42)

    # fit the model and predict
    clf = MultinomialNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    print('Multinomial prediction with tf-idf:')
    return conf_matrix, accuracy


print(prediction())
print(prediction_lemma())
print(bigram())
print(tf_idf())

Populating the interactive namespace from numpy and matplotlib
Multinomial prediction:
(array([[3636,  797],
       [ 938, 3410]]), 0.80241430361006716)
Multinomial prediction with lemmatization:
(array([[3625,  808],
       [ 852, 3496]]), 0.81095547204190865)
Multinomial prediction without stopwords:
(array([[3626,  807],
       [ 842, 3506]]), 0.81220817674524537)
top 10 indicative words for low salary class:
experience   -4.682014
nan          -4.696368
the          -4.871965
work         -4.903452
uk           -4.904317
care         -4.941719
jobs         -5.015048
role         -5.027693
client       -5.092689
team         -5.132883
dtype: float64
top 10 indicative words for high salary class:
experience   -4.457341
team         -4.875519
the          -4.884357
nan          -4.893707
uk           -4.896085
manager      -4.899771
business     -4.972877
work         -5.001205
role         -5.017079
client       -5.031225
dtype: float64
None
Multinomial prediction with bigram:
(array